In [ ]:
!pip uninstall opencv-python-headless==4.5.3.56 --yes


In [ ]:
# WandB – Install the W&B library
!pip install wandb -q
!pip install opencv-python==4.1.2.30
!pip install imageio
!pip install onnxruntime
!pip install keras2onnx
!pip install tf2onnx
!pip install pyyaml h5py


In [ ]:
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.utils import to_categorical

import wandb
from wandb.keras import WandbCallback

import tf2onnx
import onnx

import imageio, glob, cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
(train_images, train_labels),(test_images, test_labels) = keras.datasets.fashion_mnist.load_data()

# Plot head of dataset

In [ ]:
# summarize loaded dataset
# plot first few images
for i in range(5):
    # define subplot
    plt.subplot(330 + 1 + i)
    # plot raw pixel data
    plt.imshow(test_images[i], cmap=plt.get_cmap('gray'))
# show the figure
plt.show()

# Save images for API test

In [ ]:
images_to_save = test_images[:5]
labels_to_save = test_labels[:5]

In [ ]:
for i in range(len(images_to_save)):
    plt.imshow(images_to_save[i]) 
    plt.savefig('images/image_' + str(i))

# Remove saved images from test

In [ ]:
test_images = test_images[5:]
test_labels = test_labels[5:]

# Prepare data for model

In [ ]:
# Reshape the images.
train_images_re = np.expand_dims(train_images, axis=3)
test_images_re = np.expand_dims(test_images, axis=3)

classes = np.unique(train_labels)
input_shape = train_images_re[0].shape

print(classes)
print(input_shape)


# Create Model

In [ ]:
def createModel(input_shape, num_classes):
    model = Sequential([keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(num_classes)])
    return model

In [ ]:
model = createModel(input_shape, len(classes))

In [ ]:
model.summary()

In [ ]:
num_epochs=10
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])



# Train model

In [ ]:
history = model.fit(train_images_re, train_labels, epochs=num_epochs, validation_split=0.2, verbose = 1)

In [ ]:
test_loss, test_acc = model.evaluate(test_images_re,  test_labels, verbose=2)

# Save model to .onnx format

In [ ]:

onnx_model, _ = tf2onnx.convert.from_keras(model,
                                           #input_sig,
                                           opset=13)

onnx.save(onnx_model, 'models/model.onnx')

In [ ]:
model.save('models/model.h5')

In [ ]:
model1 = keras.models.load_model('models/model.h5')
model1.summary()